In [ ]:
!pip install transformers
!pip install datasets
!pip install numpy
!pip install transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 25.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 39.2 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 6.8 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-n

In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer, Trainer, TrainingArguments, EarlyStoppingCallback, RobertaConfig
from datasets import load_dataset, load_metric, Dataset, metric, concatenate_datasets
from sklearn.model_selection import train_test_split
import numpy as np
import random
import pandas as pd
import torch

In [ ]:
def tokenize(text):
    outputs = text['Outputs']
    tokenized_outputs = tokenizer(outputs, padding="max_length", truncation=True, max_length=256)
    labels = np.ones(len(outputs))  # Create an array of ones with the same length as 'Outputs'
    return {'input_ids': np.array(tokenized_outputs['input_ids']), 'labels': labels}

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

# Load the dataset
dataset = load_dataset("vedantgaur/GPTOutputs-MWP", trust_remote_code=True)

# Add a new field 'labels' that always contains the value 1 (representing 'AI generated')
dataset = dataset.map(lambda example: {'labels': 1})

# Tokenize the dataset
tokenizer = AutoTokenizer.from_pretrained("roberta-base")
tokenizedDataset = dataset.map(tokenize, batched=True)

# Split into train and test sets
trainDataset = tokenizedDataset['train']
testDataset = tokenizedDataset['validation']

# Model: roberta-base
model = AutoModelForSequenceClassification.from_pretrained("roberta-base", num_labels=2)

BertTraining_args = TrainingArguments(
    output_dir="test_trainer",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    learning_rate=5e-5,
    num_train_epochs=3,
)

# Creation of Bert Trainer Object
trainer = Trainer(
    model=model,
    args=BertTraining_args,
    train_dataset=trainDataset,
    eval_dataset=testDataset,
    compute_metrics=compute_metrics,
)

# Fine-tuning the Model
trainer.train()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/3048 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/762 [00:00<?, ? examples/s]

Map:   0%|          | 0/3048 [00:00<?, ? examples/s]

Map:   0%|          | 0/762 [00:00<?, ? examples/s]

...

Map:   0%|          | 0/762 [00:00<?, ? examples/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(
We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/tro

Step,Training Loss
500,0.002700


TrainOutput(global_step=573, training_loss=0.0023898579989662344, metrics={'train_runtime': 107.0777, 'train_samples_per_second': 85.396, 'train_steps_per_second': 5.351, 'total_flos': 1202943745105920.0, 'train_loss': 0.0023898579989662344, 'epoch': 3.0})

In [ ]:
# Predictions
metric = load_metric("accuracy")
predictions = trainer.predict(testDataset)
classPredictions = np.argmax(predictions.predictions, axis=-1)
evalMetrics = compute_metrics((predictions.predictions, predictions.label_ids))
print(evalMetrics)

apiToken = "REDACTED"

# Push the fine-tuned model to Huggingface
model.push_to_hub("SkwarczynskiP/roberta-base-finetuned-vedantgaur-AI-generated", token=apiToken)

# Push the tokenizer to Huggingface
tokenizer.push_to_hub("SkwarczynskiP/roberta-base-finetuned-vedantgaur-AI-generated", token=apiToken)

<ipython-input-4-c540b784d040>:2: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("accuracy")
/usr/local/lib/python3.10/dist-packages/datasets/load.py:759: FutureWarning: The repository for accuracy contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.0/metrics/accuracy/accuracy.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


{'accuracy': 1.0}


model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/SkwarczynskiP/roberta-base-finetuned-vedantgaur-AI-generated/commit/dded36876183e14a83687f704302c3521f3096cb', commit_message='Upload tokenizer', commit_description='', oid='dded36876183e14a83687f704302c3521f3096cb', pr_url=None, pr_revision=None, pr_num=None)